# Classifying if the Question pair is similar

In [ ]:
"""
https://www.kaggle.com/c/quora-question-pairs/data

"""

In [ ]:
import numpy as np 
import pandas as pd
import pickle
import tensorflow as tf
import tensorflow_hub as hub
# from chatterbot import ChatBot
# from chatterbot.trainers import ChatterBotCorpusTrainer
import nltk
import re


In [3]:
url='/Documents/kaggleDatasets/quora-question-pairs/train.csv'
df=pd.read_csv(url)

In [3]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [5]:
df.is_duplicate.value_counts()

0    255027
1    149263
Name: is_duplicate, dtype: int64

In [6]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)

In [73]:
def embed(input):
    return model([input])

dataset=df.head(10000).copy()
dataset['question1_vector'] = dataset.question1.map(embed)
dataset['question1_vector'] = dataset.question1_vector.map(np.array)

dataset['question2_vector'] = dataset.question2.map(embed)
dataset['question2_vector'] = dataset.question2_vector.map(np.array)
pickle.dump(dataset, open('dataset_quora.pkl', 'wb'))

In [61]:
QUESTION1_VECTORS = np.array(dataset.question1_vector)
QUESTION2_VECTORS = np.array(dataset.question2_vector)
# COSINE_THRESHOLD = 0.5
        

In [100]:
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score



In [89]:
from scipy.spatial.distance import cdist
dataset['cosine_sim']=dataset.apply(lambda x:
                                    1-cdist(x['question1_vector'],
                                       x['question2_vector'],'cosine')[0][0], axis=1 )

In [97]:
x_train, y_train, x_test, y_test=dataset[['cosine_sim']][0:8000],\
    dataset['is_duplicate'][0:8000],\
    dataset[['cosine_sim']][8000:],\
    dataset['is_duplicate'][8000:]
    
    
lr=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=42)
#Fitting the model for Bag of words
lr1=lr.fit(x_train,y_train)



In [98]:
lr_predict=lr.predict(x_test)

In [101]:
lr_report=classification_report(y_test,lr_predict,target_names=['Same','NotSame'])


In [102]:
cm=confusion_matrix(y_test,lr_predict,labels=[0,1])



In [103]:
lr_report

'              precision    recall  f1-score   support\n\n        Same       0.79      0.79      0.79      1268\n     NotSame       0.64      0.64      0.64       732\n\n    accuracy                           0.73      2000\n   macro avg       0.71      0.71      0.71      2000\nweighted avg       0.73      0.73      0.73      2000\n'

In [104]:
cm

array([[999, 269],
       [264, 468]])

In [158]:
import itertools
list2d=dataset.question1_vector
merged = list(itertools.chain(*list2d))

In [169]:
data=np.mat(merged)
print(data.shape)

(10000, 512)


In [167]:
    
lr=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=42)
#Fitting the model for Bag of words
lr1=lr.fit(data,dataset['is_duplicate'])